In [11]:
%pip install openai neo4j


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from neo4j import GraphDatabase
from openai import OpenAI

In [ ]:
username = 'neo4j'
password = ''
uri = ''
openai_api_key = ''

For example, add additional image of troubleshooting flow to IoT Manufactr page

Image URL: https://i.imgur.com/4Ni8pXo.jpeg

In [8]:
img_url= 'https://i.imgur.com/4Ni8pXo.jpeg'

In [10]:
with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    driver.execute_query(
        '''
        MATCH (w:WebPage {id: '91ce1a95-8b21-4c3c-bb3a-69c0cd294f98'})
        MATCH (wp:WebPart)-[:PART_OF]->(w) WHERE wp.order=9
        MERGE (wp)-[:LINKED_TO]->(i:Image {url: $img_url})
        RETURN i
        ''',img_url=img_url
    )


Next, get all images from the graph.

In [12]:
images = None

with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    images = driver.execute_query(
        '''MATCH (i:Image) RETURN i.url AS url
        '''
    )

In [24]:
image_links = []
for image in images.records:
    image_links.append(image['url'])

In [25]:
image_links

['https://cdn.hubblecontent.osi.office.net/m365content/publish/74f4e0ab-93a2-4d44-94d5-fda2981119c7/694075824.jpg',
 'https://cdn.hubblecontent.osi.office.net/m365content/publish/76bca189-84bb-44a3-935c-c54e7864cc9f/1273582733.jpg',
 'https://cdn.hubblecontent.osi.office.net/m365content/publish/984eb06b-e13d-4c1c-a05c-4f1f490993f1/1025633076.jpg',
 'https://cdn.hubblecontent.osi.office.net/m365content/publish/57db0400-6f15-406f-9050-559272b52e4c/1217424559.jpg',
 'https://i.imgur.com/4Ni8pXo.jpeg']

Call OpenAI

In [27]:
client = OpenAI(api_key=openai_api_key)

def get_image_description(img_url):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {
                'role': 'user',
                'content': [
                    {
                        'type': 'text',
                        'text': 'What is in this image?'
                    },
                    {
                        'type': 'image_url',
                        'image_url': {
                            'url': img_url
                        }
                    }
                ]
            }
        ],
        max_tokens=500,
    )

    return response.choices[0]

In [42]:
res = []

for url in image_links:
    desc = get_image_description(url)
    res.append({'url':url, 'description': desc.message.content})

In [43]:
for item in res:
    print(f'\n==== {item['url']} ====')
    print(item['description'])


==== https://cdn.hubblecontent.osi.office.net/m365content/publish/74f4e0ab-93a2-4d44-94d5-fda2981119c7/694075824.jpg ====
The image depicts an industrial facility, likely an oil refinery or chemical plant. It features large cylindrical storage tanks and several tall structures, which include pipes and towers, illuminated against the twilight sky. The landscape includes a field in the foreground and various buildings around the facility. The scene conveys a sense of industrial activity and infrastructure.

==== https://cdn.hubblecontent.osi.office.net/m365content/publish/76bca189-84bb-44a3-935c-c54e7864cc9f/1273582733.jpg ====
The image shows a robotic arm, likely used in industrial automation or manufacturing settings. It features a set of grippers or end effectors designed to manipulate objects. The background includes graphical elements, such as circuitry or code-like patterns, suggesting a focus on technology and automation. This creates a futuristic or high-tech atmosphere, indica

Append the description into the respective nodes in the graph

In [49]:
with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    total = driver.execute_query(
        '''
        UNWIND $res AS res
        MATCH (i:Image) WHERE i.url=res.url
        SET i.text=res.description
        RETURN COUNT(i)
        ''',res=res
    )

    print(total.records)

[<Record COUNT(i)=5>]
